In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

## Create DF to hold paper info:

In [2]:
df = pd.DataFrame(columns = ["Title", "Abstract", "Date"])
df

,Title,Abstract,Date


## Scrape the paper data given catalog URL:

In [ ]:
#catalog = "https://vtechworks.lib.vt.edu/handle/10919/19372/recent-submissions?offset=0" # CS
#catalog = "https://vtechworks.lib.vt.edu/handle/10919/5532/recent-submissions?offset=0" # Life science
catalog = "https://vtechworks.lib.vt.edu/recent-submissions?offset=0"
#catalog = "https://vtechworks.lib.vt.edu/handle/10919/5534/recent-submissions?offset=0"
#catalog = "https://vtechworks.lib.vt.edu/handle/10919/5534/discover?rpp=20&etal=0&scope=&group_by=none&page=1&sort_by=dc.date.issued_dt&order=desc&filtertype_0=department&filter_relational_operator_0=equals&filter_0=Computer+Science"
for i in range(11):
    cat_page = requests.get(catalog)
    cat_soup = BeautifulSoup(cat_page.content, "html.parser")

    # Get each URL on catalog
    div = cat_soup.find("div", class_ = "ds-static-div")
    links = div.find_all("a", class_ = lambda x: x not in ["image-link", "previous-page-link", "next-page-link"])
    for link in links:
        if 'http' in link['href']:
            continue
        url = "https://vtechworks.lib.vt.edu" + link['href']
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        abstract = soup.find("div", class_ = "line-break")
        if abstract == None:
            continue
        date = list(soup.find("div", class_ = "simple-item-view-date"))[-1]
        title = soup.find("h2", class_ = "first-page-header").text
        meta = {"Title": title, "Abstract": abstract.text, "Date": date}
        df = df.append(meta, ignore_index = True)
    num = i + 20
    old = "offset=" + str(num)
    new = "offset=" + str(num + 20)
    catalog = catalog.replace(old, new)
    time.sleep(20) # Timeout to ensure too many requests aren't sent

## Save the data to CSV:

In [ ]:
# Set desired file name
df.to_csv("Recent_papers.csv", index = False)